In [1]:
import glob 
import os
import pickle
import numpy as np
import pandas as pd
import torch
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch.distributions import Laplace
import poisevae
from poisevae.datasets import CUB
from poisevae.utils import NN_lookup, Categorical, sent_emb
from poisevae.networks.CUBNetworks import EncImg, DecImg, EncTxt, DecTxt

from scipy.linalg import eig
from sklearn.cross_decomposition import CCA

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['text.usetex'] = False

## Declarations & Loading

In [2]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
HOME_PATH = os.path.expanduser('~')
DATA_PATH = os.path.join(HOME_PATH, 'Datasets/CUB/')

In [4]:
DATA_SIZE = 81920
true_img = torch.load('../../true_data_img.pt')[:DATA_SIZE]
true_txt = torch.load('../../true_data_txt.pt')[:DATA_SIZE]
true_img_pca = torch.load('../../true_data_img_pca.pt')[:DATA_SIZE].to('cpu', torch.float32)#.numpy()
true_sent_emb = torch.load('../../true_data_sent_embedding.pt')[:DATA_SIZE].to('cpu', torch.float32)#.numpy()
sent_PC = torch.load('../../sentence_emb_PC.pt').to(device, torch.float32)
img_PC = torch.load('../../image_PC.pt').to(device, torch.float32)

In [5]:
with open(os.path.join(DATA_PATH, 'cub/oc:3_msl:32/cub.emb'), 'rb') as file:
    emb = pickle.load(file)
with open(os.path.join(DATA_PATH, 'cub/oc:3_msl:32/cub.weights'), 'rb') as file:
    weights = pickle.load(file)

In [6]:
def pca_transform(X, PC):
    li = X.split(2048, 0)
    return torch.cat([e - torch.matmul(PC, e.unsqueeze(-1)).squeeze() for e in li])

In [7]:
def calculate_corr(imgs, txts, true_img_mean, true_txt_mean):
    if isinstance(imgs, np.ndarray):
        imgs = torch.from_numpy(imgs)
    if isinstance(txts, np.ndarray):
        txts = torch.from_numpy(txts)
    if isinstance(true_img_mean, np.ndarray):
        true_img_mean = torch.from_numpy(true_img_mean)
    if isinstance(true_txt_mean, np.ndarray):
        true_txt_mean = torch.from_numpy(true_txt_mean)
    # Assume all are projected
    corr = F.cosine_similarity((imgs.cpu() - true_img_mean.cpu()), 
                               (txts.cpu() - true_txt_mean.cpu())).mean()
    return corr

In [8]:
def perform_cca(gen_img=None, gen_txt_emb=None):
    if gen_img is not None:
        gen_img = torch.cat(gen_img).to(device, torch.float32)
        gen_img = pca_transform(gen_img, img_PC.to(device, torch.float32)).cpu().numpy()
    else:
        gen_img = true_img_pca
        
    if gen_txt_emb is not None: 
        gen_txt_emb = torch.from_numpy(np.vstack(gen_txt_emb)).to(device, torch.float32)
        gen_txt_emb = pca_transform(gen_txt_emb, sent_PC.to(device, torch.float32)).cpu().numpy()
    else:
        gen_txt_emb = true_sent_emb
    
    gen_img_cca, gen_sent_cca = cca.transform(gen_img, gen_txt_emb)
    # gen_img_cca, gen_sent_cca = gen_img @ im_proj.cpu().numpy(), gen_txt_emb @ emb_proj.cpu().numpy()
    return float(calculate_corr(gen_img_cca, gen_sent_cca, true_img_cca_mean, true_sent_cca_mean))

def eval_model(vae):
    corr = []
    for mode in ('joint', 'i2s', 's2i'):
        gen_txt_emb = []
        gen_img = []
        with torch.no_grad():
            for i in range(len(true_img)):
                if mode == 'i2s':
                    results = vae([true_img[i], None], n_gibbs_iter=50)
                    gen_txt_i = results['x_rec'][1].probs.argmax(dim=1).reshape(true_img[i].shape[0], -1)
                    gen_txt_emb.append(np.zeros((true_img[i].shape[0], emb.shape[1])))
                    sent_emb(gen_txt_i.cpu().numpy().astype(np.int32), emb, weights, gen_txt_emb[-1])
                elif mode == 's2i':
                    results = vae([None, true_txt[i]], n_gibbs_iter=50)
                    gen_img.append(results['x_rec'][0].loc)
                elif mode == 'joint':
                    results = vae.generate(true_img[i].shape[0], n_gibbs_iter=50)
                    gen_img.append(results['x_rec'][0].loc)
                    gen_txt_i = results['x_rec'][1].probs.argmax(dim=1).reshape(true_img[i].shape[0], -1)
                    gen_txt_emb.append(np.zeros((true_img[i].shape[0], emb.shape[1])))
                    sent_emb(gen_txt_i.cpu().numpy().astype(np.int32), emb, weights, gen_txt_emb[-1])
                else: 
                    raise ValueError

            if mode == 'i2s':
                corr.append(('i2s', perform_cca(gen_txt_emb=gen_txt_emb)))
            elif mode == 's2i':
                corr.append(('s2i', perform_cca(gen_img=gen_img)))
            elif mode == 'joint':
                corr.append(('joint', perform_cca(gen_img=gen_img, gen_txt_emb=gen_txt_emb)))
            else: 
                raise ValueError 
        del results, gen_img, gen_txt_emb # Free memory
    return corr

In [9]:
def cca(views, k=None, eps=1e-12):
    """Compute (multi-view) CCA
    Args:
        views (list): list of views where each view `v_i` is of size `N x o_i`
        k (int): joint projection dimension | if None, find using Otsu
        eps (float): regulariser [default: 1e-12]
    Returns:
        correlations: correlations along each of the k dimensions
        projections: projection matrices for each view
    """
    V = len(views)  # number of views
    N = views[0].size(0)  # number of observations (same across views)
    os = [v.size(1) for v in views]
    kmax = np.min(os)
    ocum = np.cumsum([0] + os)
    os_sum = sum(os)
    A, B = np.zeros([os_sum, os_sum]), np.zeros([os_sum, os_sum])

    for i in range(V):
        v_i = views[i]
        v_i_bar = v_i - v_i.mean(0).expand_as(v_i)  # centered, N x o_i
        C_ij = (1.0 / (N - 1)) * torch.mm(v_i_bar.t(), v_i_bar)
        # A[ocum[i]:ocum[i + 1], ocum[i]:ocum[i + 1]] = C_ij
        B[ocum[i]:ocum[i + 1], ocum[i]:ocum[i + 1]] = C_ij
        for j in range(i + 1, V):
            v_j = views[j]  # N x o_j
            v_j_bar = v_j - v_j.mean(0).expand_as(v_j)  # centered
            C_ij = (1.0 / (N - 1)) * torch.mm(v_i_bar.t(), v_j_bar)
            A[ocum[i]:ocum[i + 1], ocum[j]:ocum[j + 1]] = C_ij
            A[ocum[j]:ocum[j + 1], ocum[i]:ocum[i + 1]] = C_ij.t()

    A[np.diag_indices_from(A)] += eps
    B[np.diag_indices_from(B)] += eps

    eigenvalues, eigenvectors = eig(A, B)
    # TODO: sanity check to see that all eigenvalues are e+0i
    idx = eigenvalues.argsort()[::-1]  # sort descending
    eigenvalues = eigenvalues[idx]  # arrange in descending order

    if k is None:
        t = threshold(eigenvalues.real[:kmax])
        k = np.abs(np.asarray(eigenvalues.real[0::10]) - t).argmin() * 10  # closest k % 10 == 0 idx
        print('k unspecified, (auto-)choosing:', k)

    eigenvalues = eigenvalues[idx[:k]]
    eigenvectors = eigenvectors[:, idx[:k]]

    correlations = torch.from_numpy(eigenvalues.real).type_as(views[0])
    proj_matrices = torch.split(torch.from_numpy(eigenvectors.real).type_as(views[0]), os)

    return correlations, proj_matrices


## CCA over truths

In [10]:
try:
    with open('CCA_model.pkl', 'rb') as f:
        cca = pickle.load(f)
except FileNotFoundError:
    cca = CCA(n_components=10, tol=1e-8)
    cca.fit(true_img_pca, true_sent_emb)
    with open('CCA_model.pkl','wb') as f:
        pickle.dump(cca, f)

In [11]:
true_img_cca, true_sent_cca = cca.transform(true_img_pca, true_sent_emb)
# corr, (im_proj, emb_proj) = cca([true_img_pca, true_sent_emb], k=40)
true_img_cca_mean, true_sent_cca_mean = true_img_cca.mean(axis=0), true_sent_cca.mean(axis=0)
calculate_corr(true_img_cca, true_sent_cca, true_img_cca_mean, true_sent_cca_mean)

tensor(0.4774, dtype=torch.float64)

In [12]:
# true_img_cca, true_sent_cca = true_img_pca @ im_proj, true_sent_emb @ emb_proj
# true_img_cca_mean, true_sent_cca_mean = true_img_cca.mean(axis=0), true_sent_cca.mean(axis=0)
# calculate_corr(true_img_cca, true_sent_cca, true_img_cca_mean, true_sent_cca_mean)

### Check on convergence

In [13]:
idx = np.arange(true_img_pca.shape[0])
np.random.shuffle(idx)
true_img_cca_hat, true_sent_cca_hat = cca.transform(true_img_pca[idx], true_sent_emb[idx])
# true_img_cca_hat, true_sent_cca_hat = true_img_pca[idx] @ im_proj, true_sent_emb[idx] @ emb_proj
calculate_corr(true_img_cca_hat, true_sent_cca_hat, true_img_cca_mean, true_sent_cca_mean)

tensor(0.4742, dtype=torch.float64)

### Check on overfitting

In [14]:
# test_true_img_pca = torch.load('true_data_img_pca.pt').cpu().numpy()[DATA_SIZE:DATA_SIZE+1000]
# test_true_sent_emb = torch.load('true_data_sent_embedding.pt').cpu().numpy()[DATA_SIZE:DATA_SIZE+1000]
# test_true_img_cca_hat, test_true_sent_cca_hat = cca.transform(test_true_img_pca, test_true_sent_emb)
# calculate_corr(test_true_img_cca_hat, test_true_sent_cca_hat, true_img_cca_mean, true_sent_cca_mean)

## CCA over trained model

### Loading checkpoints

In [15]:
condition = 'worew' # without reweighting
enc_img = EncImg(128).to(device, torch.float32)
dec_img = DecImg(128).to(device, torch.float32)
enc_txt = EncTxt(1590, 128).to(device, torch.float32)
dec_txt = DecTxt(1590, 128).to(device, torch.float32)
vae = poisevae.POISEVAE([enc_img, enc_txt], [dec_img, dec_txt], likelihoods=[Laplace, Categorical], 
                        latent_dims=[128, (128, 1, 1)], batch_size=2048, device=device)

In [16]:
true_img = true_img.to(device, torch.float32).split(2048, dim=0)
true_txt = true_txt.to(device, torch.float32).split(2048, dim=0)

In [17]:
corr = {'model': [], 'mode': [], 'correlation': []}

In [18]:
for _ in tqdm(range(5)):
    for path in glob.glob('../example/runs/CUB/worew/*'):
        try:
            vae, _, _ = poisevae.utils.load_checkpoint(vae, load_path=os.path.join(path, 'training_50.pt'))
        except FileNotFoundError:
            continue
        results = eval_model(vae)
        for mode, val in results:
            corr['model'].append('POISE-VAE')
            corr['mode'].append(mode)
            corr['correlation'].append(val)

  0%|                                                     | 0/5 [01:42<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
corr = pd.DataFrame(corr)

In [ ]:
corr.to_csv('_CCA_poise.csv')

In [ ]:
corr.groupby(['model', 'mode']).mean()

In [ ]:
corr.groupby(['model', 'mode']).std()